In [1]:
import pandas as pd
import time
from tqdm import tqdm
import os

In [3]:
assay = 'all'
rel_file = f'../maps/{assay}_related_pairs_updated.csv'
fmat_file = f'../assays/{assay}/results/plink_fmt/{assay}_rel_triangle.csv'
out = f'../assays/{assay}/results/plink_fmt/{assay}_fmat_labeled_updated_relonly.csv'

In [21]:
def read_csv_pgbar(csv_path, chunksize, dtype=object):
 
    rows = sum(1 for _ in open(csv_path, 'r')) - 1 # minus the header
    chunk_list = []
 
    with tqdm(total=rows, desc='Rows read: ') as bar:
        for chunk in pd.read_csv(csv_path, chunksize=chunksize,
                                 dtype=dtype):
            chunk_list.append(chunk)
            bar.update(len(chunk))
 
    df = pd.concat((f for f in chunk_list), axis=0)
    print('Done!')
 
    return df

In [22]:
def match_degree(x, my_dict):
    return my_dict.get(x, -1)

In [23]:
df = pd.read_csv(rel_file)
df["pair"] = [frozenset({i, j}) for i, j in zip(df["FORMATTED_REQID"], df["FORMATTED_PROBAND_REQUISITION_ID"])]

degree_dict = dict()
issues = []
dupes = []
for i in range(len(df)):
    key = df["pair"][i]
    if key not in degree_dict.keys():
        degree_dict.update({key: df["degree"][i]})
    else:
        if df["degree"][i] != degree_dict[key]:
            issues.append(key)
            degree_dict.pop(key)
        if key not in dupes:
            dupes.append(key)

In [24]:
cxf_dict = {-1: 0, 0: 1., 1: 0.5, 2: 0.25, 3: 0.125}

In [25]:
t0 = time.time()

print("Reading in features ...")
fmat = read_csv_pgbar(fmat_file, chunksize=10**6)

print("Labeling features ...")

t1 = time.time()

fmat["degree"] = [match_degree(frozenset({i, j}), degree_dict) for i, j in zip(fmat["RQ1"], fmat["RQ2"])]
fmat["coef"] = [cxf_dict[i] for i in fmat["degree"]]

print(f"Features labeled in {time.time() - t1} seconds")

print(f"Total time to completion: {time.time() - t0} seconds")

Reading in features ...


Rows read: 100%|█████████████████████████████| 2767419606/2767419606 [48:11<00:00, 956999.60it/s]


Done!
Labeling features ...


KeyError: 'RQ1'

In [ ]:
fmat.to_csv(out, index=False, chunksize=10**6)

In [ ]:
known = fmat[fmat['degree'] >= 0]
known.head()

In [ ]:
len(known)

In [26]:
del fmat